# Goals for notebook

1. **Load the data** into a pandas DataFrame

--- 

2. **Clean data** (if necessary)
    - See if there are any missing values

---

3. **Exploratory Data Analysis** (EDA)
    - Make histograms of OPS, AVG
    - See which teams had the most RBI's and HomeRuns

---
4. **Feature Engineering**
    - Make new columns describing any intersting additional statistics (for example show each player's walks to strikeout ratio)
 
---
5. **Get the data ready for a machine learning model**
    - We want to see if we can build a model to predict Home Runs and or RBI's based on other features. 